# Increase throughput for Llama-2-7b using Batching techniques on SageMaker LMI v5

---
이 노트북은 SageMaker LMI v5 컨테이너를 사용하여 Llama-2-7b-fp16-7b 대규모 언어 모델의 처리량을 증가시키기 위한 다양한 배치 기술을 탐구합니다. 이 예제에서는 LMI 컨테이너에 번들로 제공되는 DJLServing을 모델 서빙 솔루션으로 사용합니다. DJLServing은 프로그래밍 언어에 구애받지 않는 고성능 범용 모델 서빙 솔루션이며, Deep Java Library (DJL)에 의해 구동됩니다. DJL과 DJLServing에 대해 자세히 알아보려면 이 링크를 참조하세요. (https://docs.djl.ai/docs/serving/index.html)

이 노트북에서는 연속 배치를 위한 롤링 배치 기능과 페이지 주의를 제공하는 SageMaker LMI v5 컨테이너를 사용합니다. 여기서는 ml.g5.12xlarge 인스턴스에서 GPU에 걸쳐 TheBloke/Llama-2-7b-fp16-7b-dpo-v6 모델을 배포합니다.

# 1. 필요한 라이브러리 가져오기 및 SageMaker SDK 사용하여 세션 설정하기

In [ ]:
!pip install sagemaker boto3 huggingface_hub --upgrade --quiet

In [ ]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

# 2. Hugging Face LLM DLC 검색

일반적인 Hugging Face 모델을 배포하는 것과 비교할 때, 우선 컨테이너 URI를 검색하고 이를 HuggingFaceModel 모델 클래스에 image_uri를 지정하여 제공해야 합니다. Amazon SageMaker에서 새로운 Hugging Face LLM DLC를 검색하려면, SageMaker SDK에서 제공하는 get_huggingface_llm_image_uri 메소드를 사용할 수 있습니다. 이 메소드를 통해 지정된 백엔드, 세션, 지역, 버전에 기반한 원하는 Hugging Face LLM DLC의 URI를 검색할 수 있습니다. 사용 가능한 버전은 여기에서 확인할 수 있습니다.

In [ ]:
from sagemaker.huggingface import get_huggingface_llm_image_uri
from sagemaker import image_uris

# retrieve the llm image uri
llm_image = image_uris.retrieve(
    framework="djl-deepspeed", region=sess.boto_region_name, version="0.25.0"
)

# print ecr image uri
print(f"llm image uri: {llm_image}")

# You can retrieve the image in this way:
# llm_image = get_huggingface_llm_image_uri(
#   backend="lmi", # or huggingface
#   region=sess.boto_region_name
# )

# # print ecr image uri
# print(f"llm image uri: {llm_image}")

### 허깅페이스에서 모델을 다운로드하고 Amazon S3에 모델 아티팩트를 업로드합니다.

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path
import os

# - This will download the model into the current directory where ever the jupyter notebook is running
local_model_path = Path(".")
local_model_path.mkdir(exist_ok=True)
model_name = "TheBloke/Llama-2-7b-fp16"
# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.txt", "*.model", "*.safetensors", "*.bin", "*.chk", "*.pth"]

# - Leverage the snapshot library to donload the model since the model is stored in repository using LFS
model_download_path = snapshot_download(
    repo_id=model_name, cache_dir=local_model_path, allow_patterns=allow_patterns
)

In [ ]:
# upload files from local to S3 location
s3_model_prefix = "hf-large-model-djl/Llama-2-7b-fp16/model"  # folder within bucket where code artifact will go
pretrained_model_location = sess.upload_data(path=model_download_path, key_prefix=s3_model_prefix)
print(f"Model uploaded to --- > {pretrained_model_location}")

# 3. Amazon SageMaker에 Llama-2-7b-fp16 배포하기

Amazon SageMaker에 파인튜닝한 모델을 배포하기 위해 `HuggingFaceModel` 모델 클래스를 생성하고, `hf_model_id`, `instance_type` 등을 포함한 엔드포인트 구성을 정의합니다. 여기서는 4개의 NVIDIA A10G GPU와 96GB의 GPU 메모리를 갖춘 `g5.12xlarge` 인스턴스 타입을 사용할 것입니다.

먼저 serving.properties for Paged Attention Batching 파일을 생성합니다.

In [ ]:
!rm -rf code_Llama-2-7b-fp16
!mkdir -p code_Llama-2-7b-fp16

In [ ]:
%%writefile code_Llama-2-7b-fp16/serving.properties
engine = MPI
option.tensor_parallel_degree = 1
option.rolling_batch = auto
option.max_rolling_batch_size = 8
option.model_loading_timeout = 3600
option.model_id = {{model_id}}
option.paged_attention = true
option.trust_remote_code = true
option.dtype = fp16
option.enable_streaming=True

In [ ]:
import jinja2
from pathlib import Path

jinja_env = jinja2.Environment()

# we plug in the appropriate model location into our `serving.properties`
template = jinja_env.from_string(Path("code_Llama-2-7b-fp16/serving.properties").open().read())
Path("code_Llama-2-7b-fp16/serving.properties").open("w").write(
    template.render(model_id=pretrained_model_location)
)
!pygmentize code_Llama-2-7b-fp16/serving.properties | cat -n

In [ ]:
!rm model.tar.gz
!tar czvf model.tar.gz code_Llama-2-7b-fp16

In [ ]:
s3_code_prefix = "hf-large-model-djl/Llama-2-7b-fp16/code"  # folder within bucket where code artifact will go
s3_code_artifact = sess.upload_data("model.tar.gz", sess.default_bucket(), s3_code_prefix)

In [ ]:
sm_client = boto3.client("sagemaker")

from sagemaker.utils import name_from_base

model_name = name_from_base(f"Llama-2-7b-fp16-mpi")
print(model_name)

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={"Image": llm_image, "ModelDataUrl": s3_code_artifact},
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

In [ ]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.2xlarge",
            "InitialInstanceCount": 1,
            "ModelDataDownloadTimeoutInSeconds": 900,
            "ContainerStartupHealthCheckTimeoutInSeconds": 900,
        },
    ],
)
endpoint_config_response

In [ ]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

### 시간이 걸릴 수 있으니 잠시 기다립니다.

In [ ]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

## 4. 배포된 모델에서 추론 요청하기

이 코드를 사용하여 자신의 입력으로 모델을 테스트할 수도 있습니다.


In [ ]:
from sagemaker import serializers, deserializers

endpoint_name="Llama-2-7b-fp16-mpi-XXX-endpoint"

llm = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=serializers.JSONSerializer(),
    deserializer=deserializers.JSONDeserializer(),
)

In [ ]:
llm.predict(
    {
        "inputs": ["Amazon.com is the best ", "Large model inference is"],
        "parameters": {"min_new_tokens": 20, "max_new_tokens": 200, "temperature": 0.8},
    }
)

## 5. Benchmarking throughput for Paged Attention Batching

In [ ]:
!wget https://github.com/frankfliu/junkyard/releases/download/v0.3.1/awscurl
!chmod +x awscurl

In [ ]:
!rm -rf dyn_prompts
!mkdir dyn_prompts
!echo '{"inputs":"The diamondback terrapin or simply terrapin is a species of turtle native to the brackish coastal tidal marshes of the","parameters":{"min_new_tokens":128, "max_new_tokens":128, "do_sample":true}}' > dyn_prompts/dyn_prompt1.txt
!echo '{"inputs":"Write a program to add two numbers in python","parameters":{"min_new_tokens":128, "max_new_tokens":128, "do_sample":true}}' > dyn_prompts/dyn_prompt2.txt
!echo '{"inputs":"Generate a list of ten titles for my book. The book is about my travels around the world, experiencing different cultures and cuisines, meeting many different personalities and finally settling in a remote village in the himalayas","parameters":{"min_new_tokens":128, "max_new_tokens":128, "do_sample":true}}' > dyn_prompts/dyn_prompt3.txt

In [ ]:
!apt -y update
!apt -y -qq install default-jre wget 

In [ ]:
!TOKENIZER=TheBloke/Llama-2-7b-fp16 AWS_REGION=us-east-1 ./awscurl --region us-east-1 -c 10 -N 10 -X POST "https://runtime.sagemaker.us-east-1.amazonaws.com/endpoints/Llama-2-7b-fp16-mpi-XXX/invocations" --connect-timeout 60   -H "Content-Type: application/json" --dataset dyn_prompts -t -n sagemaker